In [22]:
import pandas as pd


In [23]:
def read_markdown_table(md_file_path):
    # Read the markdown file
    with open(md_file_path, 'r') as file:
        lines = file.readlines()
    
    # Initialize an empty list to hold rows of the table
    table_data = []
    
    # Process each line
    for line in lines:
        # Check if the line is part of the table
        if '|' in line:
            # Split the line by '|' to get individual cell values
            row = line.strip().split('|')
            # Filter out empty strings and strip whitespace from each cell
            row = [cell.strip() for cell in row if cell.strip()]
            # Add the row to the table data list
            table_data.append(row)
    
    # The first row is the header
    headers = table_data[0]
    # The rest are the data rows
    data_rows = table_data[2:]
    
    # Create a pandas DataFrame
    df = pd.DataFrame(data_rows, columns=headers)

    # replace "-" with NaN
    df = df.replace('-', pd.NA)
    
    # convert all columns containing "Time" to float
    for col in df.columns:
        if 'Time' in col:
            df[col] = df[col].str.extract(r'(\d+\.\d+)').astype(float)
            
    return df

# Example usage
md_file_path = 'README.md'
df = read_markdown_table(md_file_path)


In [24]:
# to long format, wrap columns containing Time
df = df.melt(id_vars=['Cluster', 'Dataset', "Benchmark", "Device Name", "# Devices", "Num Workers"], var_name='Measurement', value_name='Time')
print(df)

   Cluster   Dataset         Benchmark Device Name # Devices Num Workers  \
0       TN  ImageNet         Inference         A40         4           4   
1       TN  ImageNet             Train         A40         4           4   
2       TN  ImageNet  Smooth Inference         A40         4           4   
3       TN     Dummy         Inference         A40         4           4   
4       TN     Dummy             Train         A40         4           4   
..     ...       ...               ...         ...       ...         ...   
67   JADE2  ImageNet             Train         A40         4           4   
68   JADE2  ImageNet  Smooth Inference         A40         4           4   
69   JADE2     Dummy         Inference         A40         4           4   
70   JADE2     Dummy             Train         A40         4           4   
71   JADE2     Dummy  Smooth Inference         A40         4           4   

          Measurement    Time  
0   Data Loading Time  0.0649  
1   Data Loading Time  

In [25]:
# get mean for benchmark, cluster, device and Measurement
mean_times = df.groupby(['Cluster', "Benchmark", "Device Name", "Measurement"])["Time"].mean().reset_index()

# pivot to wide format. keep rows: Benchmark, Device. columns: Cluster and Measurement
mean_times = mean_times.pivot_table(index=["Benchmark", "Device Name"], columns=["Measurement", "Cluster"], values="Time").reset_index()
mean_times


Measurement         Benchmark Device Name Backward Time           \
Cluster                                           JADE2       TN   
0                   Inference         A40           NaN      NaN   
1                   Inference        V100           NaN      NaN   
2            Smooth Inference        V100           NaN      NaN   
3                       Train         A40        0.7074  0.20220   
4                       Train        V100           NaN  0.24425   

Measurement Data Loading Time          Data Processing Time Forward Time  \
Cluster                 JADE2       TN                   TN        JADE2   
0                     4.58895  0.05670                  NaN      1.03845   
1                         NaN  0.07775                  NaN          NaN   
2                         NaN  0.10155               0.0009          NaN   
3                     4.05865  0.03245                  NaN      1.21780   
4                         NaN  0.02745                  NaN          NaN   

Measurement           
Cluster           TN  
0            0.11120  
1            0.11760  
2            0.00795  
3            0.12915  
4            0.12695